Start up an AWS ubuntu instance and install docker. For this analysis, the following specs should be more than enough to run the bioinformatics pipeline.  





In [0]:
docker build  .
docker push


Pull the docker image.

In [0]:
sudo docker pull jonessarae/nlm_workshop:seq_tools

Download SRA files with sra_download.sh (check permissions) Already hardcoded with the SRA accession number. 


In [0]:
./sra_download.sh

Run the docker container and add home directory to container. The rest of this notebook will be done within the container.

---



In [0]:
sudo docker run -it -v ~:/mnt jonessarae/nlm_workshop:seq_tools

Convert SRA files into FASTQ files and split the files into read 1 and read 2. The command fastq-dump for SRA-toolkit does not use multi-threading. Better to find another package that can multi-thread. Can avoid splitting? In BWA can use -p option. How about others?

In [0]:
fastq-dump --split-files --origfmt --gzip SRR2989954.sra
gunzip SRR2989954_1.fastq.gz SRR2989954_1.fastq.gz


Download reference genome hg19.2bit and tool twoBitToFa, convert 2bit file to fasta file, and index it with bwa. Use -t to add more cores for running bwa index faster.

In [0]:
wget http://hgdownload.cse.ucsc.edu/goldenPath/hg19/bigZips/hg19.2bit

rsync -aP rsync://hgdownload.soe.ucsc.edu/genome/admin/exe/linux.x86_64/twoBitToFa
chmod 744 twoBitToFa
./twoBitToFa hg19.2bit hg19.fa

bwa index -a bwtsw hg19.fa 

Download reference genome hg19 annotation file (GTF). There are two files to choose from. Not sure which to use. 
https://www.gencodegenes.org/releases/19.html

In [0]:
wget ftp://ftp.ebi.ac.uk/pub/databases/gencode/Gencode_human/release_19/gencode.v19.annotation.gtf.gz


Exome: Align paired reads to the reference genome and convert it to a bam file. With an AWS instance with 96 cores, this step took ~25 minutes (on file 54) with 96 threads. 

In [0]:
bwa mem -t 8 ref/hg19.fa SRR2989954_1.fastq SRR2989954_2.fastq | samtools view -Sb - > SRR2989954.bam


Sort the bam files by genomic order.


In [0]:
samtools sort -@ 4 SRR2989954.bam -o SRR2989954_sorted.bam
samtools sort -@ 8 SRR2989963.bam -o SRR2989963_sorted.bam

RNA: Make genome indices that STAR requires. Always ensure read/write priveleges on files/folders. Check what read length should be: ReadLength-1. Does it matter if not 99? Change number of threads to 8. Mention spec requirements.

In [0]:
mkdir star_indices
STAR --runThreadN 4 --runMode genomeGenerate --genomeDir star_indices --genomeFastaFiles ref/hg19.fa --sjdbGTFfile gtf//gencode.v19.annotation.gtf --sjdbOverhang 100


RNA: Align paired reads to the reference genome. Don't forget to make a mapped_rna directory. 

In [0]:
mkdir mapped_rna
STAR --readFilesIn SRR2989969_1.fastq SRR2989969_2.fastq --runThreadN 8 --genomeDir star_indices --outFileNamePrefix mapped_rna/SRR29899692pass --genomeLoad NoSharedMemory --sjdbGTFfile gtf/gencode.v19.annotation.gtf --outSAMtype BAM SortedByCoordinate --twopassMode Basic


Exome: GATK4 from bioconda.

In [0]:
java -Xmx4g -jar GenomeAnalysisTK.jar -I SRR2989954_sorted.bam -R ref/hg19.fa -T RealignerTargetCreator -o SRR2989954.intervals –known Mills_and_1000G_gold_standard.indels.hg19.vcf --read_filter MappingQualityZero 
java -Xmx4g -jar GenomeAnalysisTK.jar -I SRR2989954_sorted.bam -R ref/hg19.fa -T IndelRealigner -targetIntervals sample.intervals -o SRR2989954_realign.bam -known Mills_and_1000G_gold_standard.indels.hg19.vcf --read_filter MappingQualityZero

java -Xmx4g -jar GenomeAnalysisTK.jar -I SRR2989963_sorted.bam -R ref/hg19.fa -T RealignerTargetCreator -o SRR2989954.intervals –known Mills_and_1000G_gold_standard.indels.hg19.vcf --read_filter MappingQualityZero 
java -Xmx4g -jar GenomeAnalysisTK.jar -I SRR2989963_sorted.bam -R ref/hg19.fa -T IndelRealigner -targetIntervals SRR2989963.intervals -o SRR2989963_realign.bam -known Mills_and_1000G_gold_standard.indels.hg19.vcf --read_filter MappingQualityZero


Filter out reads that have mapping quality of < 20.

In [0]:
samtools view -b -q 20 SRR2989954_realign.bam > SRR2989954_m20.bam
samtools view -b -q 20 SRR2989963_realign.bam > SRR2989963_m20.bam
samtools view -b -q 20 mapped_rna/SRR29899692passAligned.sortedByCoord.out.bam > SRR2989969_m20.bam

Deduplicate files. We installed picard separately, but picard is also part of GATK4. 

In [0]:

java -jar picard.jar MarkDuplicates I=SRR2989954_m20.bam O=SRR2989954_m20dedup.bam REMOVE_DUPLICATES=true METRICS_FILE=metrics.txt
java -jar picard.jar MarkDuplicates I=SRR2989963_m20.bam O=SRR2989963_m20dedup.bam REMOVE_DUPLICATES=true METRICS_FILE=metrics.txt
picard MarkDuplicates I=SRR2989969_m20.bam O=SRR2989969_m20dedup.bam REMOVE_DUPLICATES=true METRICS_FILE=metrics.txt



Generate VCF for multiple BAM files. Not sure how they run this except that it outputs vcf format. Not sure where the normal sample is in the VCF file. Assumed they combined all three with normal.

In [0]:
samtools mpileup -f hg.19a SRR2989954_m20dedup.bam SRR2989963_m20dedup.bam SRR2989969_m20dedup.bam -v -o p1_ov.vcf

Calls variants from a mpileup dataset and produces a VCF.



In [0]:
varscan mpileup2snp p1_ov.vcf --min-coverage 5 --min-reads2 0 --min-avg-qual 20 --min-var-freq 0 --output-vcf 13


Run snpEff, a variant annotation and effect prediction tool. It annotates and predicts the effects of variants on genes. Not sure which version of the GRCh37 database was used. 

In [0]:
#snpEff databases | grep -i sapiens
snpEff download GRCh37.75
snpEff GRCh37.75 p1_ov.vcf > p1_ov.ann.vcf


Count total reads and compare to supplementary figure 1.

In [0]:
samtools flagstat <sample.bam>